# FastAPI Docs RAG

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Get all Tutorials URLs from FastAPI Learn

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_fastapi_docs_urls_requests():
    base_url = "https://fastapi.tiangolo.com/learn/"
    print("Carregando a página com a biblioteca 'requests'...")
    try:
        response = requests.get(base_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')
    except requests.exceptions.RequestException as e:
        print(f"Erro ao carregar a página: {e}")
        return []

    # Encontra o menu de navegação principal
    nav_container = soup.find("nav", {"class": "md-nav--primary"})
    if not nav_container:
        print("Falha: Menu de navegação primário não encontrado.")
        return []

    urls = set() # Usar um set para evitar duplicatas automaticamente

    # Itera sobre todas as seções do tutorial e guia avançado
    sections_to_find = ["Tutorial - User Guide", "Advanced User Guide"]
    for section_text in sections_to_find:
        span = nav_container.find("span", class_="md-ellipsis", string=lambda t: t and section_text in t.strip())
        if span:
            # Navega até o <li> pai e encontra todos os links dentro dele
            container = span.find_parent('li')
            if container:
                for item in container.find_all("a", class_="md-nav__link"):
                    href = item.get("href")
                    if href:
                        full_url = urljoin("https://fastapi.tiangolo.com/", href)
                        urls.add(full_url)

    unique_urls = sorted(list(urls)) # Converte para lista e ordena
    print(f"Encontradas {len(unique_urls)} URLs únicas para carregar.")
    return unique_urls

In [3]:
urls_to_load = get_fastapi_docs_urls_requests()
urls_to_load

Carregando a página com a biblioteca 'requests'...
Encontradas 78 URLs únicas para carregar.


['https://fastapi.tiangolo.com/advanced/',
 'https://fastapi.tiangolo.com/advanced/additional-responses/',
 'https://fastapi.tiangolo.com/advanced/additional-status-codes/',
 'https://fastapi.tiangolo.com/advanced/advanced-dependencies/',
 'https://fastapi.tiangolo.com/advanced/async-tests/',
 'https://fastapi.tiangolo.com/advanced/behind-a-proxy/',
 'https://fastapi.tiangolo.com/advanced/custom-response/',
 'https://fastapi.tiangolo.com/advanced/dataclasses/',
 'https://fastapi.tiangolo.com/advanced/events/',
 'https://fastapi.tiangolo.com/advanced/generate-clients/',
 'https://fastapi.tiangolo.com/advanced/middleware/',
 'https://fastapi.tiangolo.com/advanced/openapi-callbacks/',
 'https://fastapi.tiangolo.com/advanced/openapi-webhooks/',
 'https://fastapi.tiangolo.com/advanced/path-operation-advanced-configuration/',
 'https://fastapi.tiangolo.com/advanced/response-change-status-code/',
 'https://fastapi.tiangolo.com/advanced/response-cookies/',
 'https://fastapi.tiangolo.com/advanc

## Load Content of URLs

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer(class_=("md-content"))
loader = WebBaseLoader(
    web_paths=urls_to_load,
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
for i in range(len(docs)):
    print(f"Total characters: {len(docs[i].page_content)}")

Total characters: 674
Total characters: 10751
Total characters: 5516
Total characters: 9209
Total characters: 4353
Total characters: 11530
Total characters: 14585
Total characters: 5578
Total characters: 9841
Total characters: 14593
Total characters: 4040
Total characters: 12133
Total characters: 3117
Total characters: 12194
Total characters: 1793
Total characters: 2542
Total characters: 3513
Total characters: 2351
Total characters: 582
Total characters: 12940
Total characters: 270213
Total characters: 14267
Total characters: 3181
Total characters: 3734
Total characters: 10078
Total characters: 788
Total characters: 1088
Total characters: 2276
Total characters: 28049
Total characters: 1295
Total characters: 4850
Total characters: 9423
Total characters: 23812
Total characters: 8239
Total characters: 17301
Total characters: 19557
Total characters: 16247
Total characters: 9947
Total characters: 6397
Total characters: 3481
Total characters: 5121
Total characters: 2230
Total characters: 175

In [6]:
len(docs)

78

## Indexing Data

### Splitting in Chunks

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)
split_docs = splitter.split_documents(docs)
len(split_docs)

1949

### Embedding Model

In [8]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

llm_embeddings = GoogleGenerativeAIEmbeddings(model='models/gemini-embedding-001')

### Vector Store

In [9]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="fastapi_docs",
    embedding_function=llm_embeddings,
    persist_directory="./chroma_fastapi_db",
)

In [10]:
import time

def add_documents_in_batches(vector_store, documents, batch_size=100, delay_seconds=20):
    """
    Adiciona documentos ao vector_store em lotes para evitar erros de limite de taxa.
    """
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]

        print(f"Processando lote de {i} a {i + len(batch)}...")

        vector_store.add_documents(batch)

        print(f"Lote processado. Aguardando {delay_seconds} segundos...")

        # Pausa a execução para respeitar o limite da API
        time.sleep(delay_seconds)

try:
    add_documents_in_batches(vector_store, split_docs, 20, 35)
    print("Todos os documentos foram adicionados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante o processamento: {e}")

Processando lote de 0 a 20...
Lote processado. Aguardando 35 segundos...
Processando lote de 20 a 40...
Lote processado. Aguardando 35 segundos...
Processando lote de 40 a 60...
Lote processado. Aguardando 35 segundos...
Processando lote de 60 a 80...
Lote processado. Aguardando 35 segundos...
Processando lote de 80 a 100...
Lote processado. Aguardando 35 segundos...
Processando lote de 100 a 120...
Lote processado. Aguardando 35 segundos...
Processando lote de 120 a 140...
Lote processado. Aguardando 35 segundos...
Processando lote de 140 a 160...
Lote processado. Aguardando 35 segundos...
Processando lote de 160 a 180...
Lote processado. Aguardando 35 segundos...
Processando lote de 180 a 200...
Lote processado. Aguardando 35 segundos...
Processando lote de 200 a 220...
Lote processado. Aguardando 35 segundos...
Processando lote de 220 a 240...
Lote processado. Aguardando 35 segundos...
Processando lote de 240 a 260...
Lote processado. Aguardando 35 segundos...
Processando lote de 26

## ChatModel

In [21]:
from langchain.chat_models import init_chat_model

llm_flash = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
llm_pro = init_chat_model("gemini-2.5-pro", model_provider="google_genai")

## Retrieval Chain

### Gemini Flash

In [23]:
from langchain.chains.retrieval_qa.base import RetrievalQA

qa_chain_flash = RetrievalQA.from_chain_type(llm_flash, retriever=vector_store.as_retriever())

pergunta = "How can i add security, authentication and authorization in my API?"
resultado = qa_chain_flash.invoke({ "query" : pergunta})
print(resultado)

{'query': 'How can i add security, authentication and authorization in my API?', 'result': 'FastAPI provides several tools to help you deal with Security easily, rapidly, and in a standard way.\n\nHere are some of the ways to handle security, authentication, and authorization mentioned:\n\n*   **OAuth2**: A specification defining several ways to handle authentication and authorization. It includes various "flows," with the `password` flow being perfectly suitable for handling authentication directly within the same application.\n*   **apiKey**: An application-specific key that can be passed from:\n    *   A query parameter\n    *   A header\n    *   A cookie\n*   **http**: Standard HTTP authentication systems, including:\n    *   `bearer`: A `Authorization` header with a value of `Bearer` plus a token (inherited from OAuth2).\n    *   HTTP Basic authentication.\n    *   HTTP Digest, etc.\n*   **openIdConnect**: A way to define how to discover OAuth2 authentication data automatically.\n

### Gemini Pro

In [24]:
from langchain.chains.retrieval_qa.base import RetrievalQA

qa_chain_pro = RetrievalQA.from_chain_type(llm_pro, retriever=vector_store.as_retriever())

pergunta = "How can i add security, authentication and authorization in my API?"
resultado = qa_chain_pro.invoke({ "query" : pergunta})
print(resultado)

{'query': 'How can i add security, authentication and authorization in my API?', 'result': 'Based on the provided context, FastAPI offers several tools to help you add security, authentication, and authorization in a standard way. Here\'s a summary of the approaches mentioned:\n\n*   **OAuth2:** This is a specification for handling authentication and authorization. FastAPI can integrate with it, particularly using the `password` "flow," which is well-suited for handling authentication with a username and password directly within your application.\n*   **apiKey:** You can use an application-specific key that can be sent in a query parameter, a header, or a cookie.\n*   **http:** FastAPI supports standard HTTP authentication systems, including:\n    *   **bearer:** Using an `Authorization` header with a `Bearer` token.\n    *   **HTTP Basic authentication.**\n*   **OpenID Connect:** This can be used to automatically discover OAuth2 authentication data.\n\nThe context suggests that if you